# Mutation spectrum and rates at four-fold synonymous sites
Analyze the mutation rates and spectrum only at four-fold synonymous sites, as defined in relation to the founder sequence for that clade.

Import Python modules:

In [1]:
import itertools
import os

import altair as alt

import numpy

import pandas as pd

import yaml

import scipy

import sklearn.decomposition

Get input variables from `snakemake`:

In [2]:
if "snakemake" in locals() or "snakemake" in globals():
    mutation_counts_csv = snakemake.input.mutation_counts_csv
    clade_founder_nts_csv = snakemake.input.clade_founder_nts_csv
    rates_by_clade_csv = snakemake.output.rates_by_clade
    rates_plot = snakemake.output.rates_plot
    synonymous_spectra_min_counts = snakemake.params.synonymous_spectra_min_counts
    subset_order = list(snakemake.params.sample_subsets)
    clade_synonyms = snakemake.params.clade_synonyms
else:
    # if running interactively
    mutation_counts_csv = "../results/mutation_counts/aggregated.csv"
    clade_founder_nts_csv = "../results/clade_founder_nts/clade_founder_nts.csv"
    rates_by_clade_csv = "../results/synonymous_mut_rates/rates_by_clade.csv"
    rates_plot = "../results/synonymous_mut_rates/rates_plot.html"
    with open("config.yaml" if os.path.isfile("config.yaml") else "../config.yaml") as f:
        config = yaml.safe_load(f)
    synonymous_spectra_min_counts = config["synonymous_spectra_min_counts"]
    subset_order = list(config["sample_subsets"])
    clade_synonyms = config["clade_synonyms"]

Read the mutation counts and assign mutation types:

In [3]:
mutation_counts = pd.read_csv(mutation_counts_csv).assign(
    mut_type=lambda x: x["nt_mutation"].map(lambda m: f"{m[0]}to{m[-1]}")
)

For each clade plot the top mutations as a fraction of all mutations in that clade, just using the "all" subset.

You can mouseover points to highlight mutations (which will highlight all mutations at that site on all facets), and click the legend to show/hide excluded or non-excluded mutations.

This plot is useful to look at to identifier apparent outlier sites with aberrantly high mutation counts that can then be specified for exclusion (note those specifications are done in the pipeline `config.yaml` file, and also all reversions from clade founder to reference may be excluded):

In [4]:
top_n = 100  # plot this many per clade

mutation_freqs = (
    mutation_counts.query("subset == 'all'")
    .sort_values(["clade", "count"], ascending=False)
    .groupby("clade")
    .head(n=top_n)
    .assign(
        freq=lambda x: x["count"] / x.groupby("clade")["count"].transform("sum"),
        rank=lambda x: x.groupby("clade")["freq"].rank(ascending=False, method="first"),
        exclude=lambda x: x["exclude"].map({True: "yes", False: "no"}),
    )
)

select_exclude = alt.selection_multi(
    fields=["exclude"],
    bind="legend",
    init=[{"exclude": "yes"}, {"exclude": "no"}],
)

select_site = alt.selection_single(
    fields=["nt_site"],
    on="mouseover",
    empty="none",
)

mutation_freqs_chart = (
    alt.Chart(mutation_freqs)
    .encode(
        x="rank",
        y="freq",
        strokeWidth=alt.condition(select_site, alt.value(2), alt.value(0)),
        color=alt.Color("exclude", scale=alt.Scale(domain=["yes", "no"])),
        shape=alt.Shape("synonymous"),
        size=alt.condition(select_site, alt.value(50), alt.value(25)),
        tooltip=["nt_site", "nt_mutation", "count", "freq"],
    )
    .mark_point(filled=True, stroke="black")
    .properties(width=200, height=100)
    .facet("clade", columns=4)
    .add_selection(select_exclude, select_site)
    .transform_filter(select_exclude)
)

mutation_freqs_chart

alt.FacetChart(...)

Tally mutation type counts among **only four-fold synonymous** mutations for each clade and subset, also removing any mutations specified for exclusion:

In [5]:
mut_type_counts = (
    mutation_counts.query("synonymous")
    .query("four_fold_degenerate")
    .query("not exclude")
    .groupby(["clade", "subset", "mut_type"], as_index=False)
    .aggregate({"count": "sum"})
)

Now also repeat these mutation type counts tally, but any mutations in the top 10 most frequent observed mutation for any clade, not doing any subsetting (just taking subset "all"):

In [6]:
exclude_top_n = 5  # exclude mutations in this top rank for any clade

mut_type_counts_exclude_top = (
    mutation_counts.query("synonymous")
    .query("four_fold_degenerate")
    .query("not exclude")
    .query("subset == 'all'")
    .assign(
        clade_rank=lambda x: x.groupby("clade")["count"].rank(
            ascending=False, method="min"
        ),
        highest_rank=lambda x: x.groupby("nt_mutation")["clade_rank"].transform("min"),
    )
    .query("highest_rank > @exclude_top_n")
    .groupby(["clade", "mut_type"], as_index=False)
    .aggregate({"count": "sum"})
)

Plot total mutation counts for each clade and subset on a log scale.
Also draw a line at our minimum cutoff: we only keep subsets above this cutoff:

In [7]:
clade_counts = mut_type_counts.groupby(["clade", "subset"], as_index=False).aggregate(
    {"count": "sum"}
)

clade_counts_chart = (
    alt.Chart(clade_counts)
    .encode(
        x="clade",
        y=alt.Y("count", title="total mutations", scale=alt.Scale(type="log")),
        tooltip=["clade", "subset", "count"],
        color="subset",
    )
    .mark_circle(size=50, opacity=0.7)
    .properties(width=alt.Step(18), height=175)
)

# draw cutoff line
cutoff = (
    alt.Chart(pd.DataFrame({"y": [synonymous_spectra_min_counts]}))
    .encode(y="y")
    .mark_rule(strokeDash=[2, 2])
)

(clade_counts_chart + cutoff).configure_axis(grid=False)

alt.LayerChart(...)

For genome partitioning, we subdivide the genome into halves based on the first and last site with an observed mutation:

In [8]:
n_partitions = 2

min_site = mutation_counts["nt_site"].min()
max_site = mutation_counts["nt_site"].max() + 1
partition_bounds = numpy.linspace(min_site, max_site, n_partitions + 1)


def assign_partition(r):
    """Assign nucleotide mutation to its partition."""
    for i in range(1, n_partitions + 1):
        if partition_bounds[i - 1] <= r < partition_bounds[i]:
            return f"partition {i}"


mutation_counts = mutation_counts.assign(
    partition=lambda x: x["nt_site"].map(assign_partition)
)

Get PCA of mutation spectrum, using only filtered synonymous mutation counts for non-excluded mutations for clades/subsets/partitions with adequate counts.

We do the PCA on four different ways of partitioning the data:

 1. Just looking at the "all" subset for each clade across entire genome.
 2. Looking at the "all" subset for each clade across entire genome, but excluding any mutation that is among the top ranked most counts for any clade.
 3. Looking at all subsets for each clade across entire genome.
 4. Looking at the "all" subset along partitions of the genome. 

We standardize the vectors of mutation fractions before doing the PCA.

There is also an option, currently not used, to do a composite log ratio (CLR) transform the fraction of all mutations that are each type before performing the PCA, using the formula [here](https://www.rdocumentation.org/packages/compositions/versions/2.0-4/topics/clr) (see [this paper](https://www.jstor.org/stable/2335943)).

In the plots below, you can mouseover the points for details and click on clades in legends (shift click for multiple clades) to highlight just points for the selected clade(s).
You can also use the scroll bar to only show points with at least the indicated number of total synonymous mutation counts (after filtering):

In [9]:
def clr(frac_vec):
    """https://www.rdocumentation.org/packages/compositions/versions/2.0-4/topics/clr"""
    assert all(frac_vec > 0), "does not currently handle zeros in frac_vec"
    return numpy.log(frac_vec) - 1 / len(frac_vec) * numpy.log(frac_vec).sum()


for title, subsets, partition, exclude_top, clr_transform in [
    ("all samples, whole genome", ["all"], False, False, False),
    ("all samples, whole genome, +/- top mutations", ["all"], False, True, False),
    ("by region, whole genome", subset_order, False, False, False),
    ("all samples, partitioned genome", ["all"], True, False, False),
]:

    filtered_mutation_counts = (
        mutation_counts.query("synonymous")
        .query("four_fold_degenerate")
        .query("not exclude")
        .query("subset in @subsets")
    )

    if partition:
        filtered_mutation_counts = pd.concat(
            [
                filtered_mutation_counts.assign(partition="all"),
                filtered_mutation_counts,
            ]
        )
    else:
        filtered_mutation_counts = filtered_mutation_counts.assign(partition="all")

    mut_type_counts = filtered_mutation_counts.groupby(
        ["clade", "subset", "partition", "mut_type"], as_index=False
    ).aggregate({"count": "sum"})

    if exclude_top:
        assert all(mut_type_counts["partition"] == "all")
        assert all(mut_type_counts["subset"] == "all")
        mut_type_counts = pd.concat(
            [
                mut_type_counts.assign(excluded="no"),
                mut_type_counts_exclude_top.assign(
                    partition="all",
                    subset="all",
                    excluded="yes",
                ),
            ]
        )
    else:
        mut_type_counts = mut_type_counts.assign(excluded="no")

    mut_type_freqs = mut_type_counts.assign(
        total_count=lambda x: (
            x.groupby(["clade", "subset", "partition", "excluded"])["count"].transform(
                "sum"
            )
        ),
        freq=lambda x: x["count"] / x["total_count"],
    ).query("total_count >= @synonymous_spectra_min_counts")

    mut_type_freqs = mut_type_freqs.pivot_table(
        index=["clade", "subset", "partition", "excluded", "total_count"],
        values="freq",
        columns="mut_type",
        fill_value=0,
    )

    if clr_transform:
        display(mut_type_freqs)
        mut_type_freqs = mut_type_freqs.apply(clr, axis=1)
        display(mut_type_freqs)

    scaled_freqs = sklearn.preprocessing.StandardScaler().fit_transform(
        mut_type_freqs.values
    )
    pca = sklearn.decomposition.PCA(n_components=2)
    pca_coords = pca.fit_transform(scaled_freqs)
    assert len(pca_coords) == len(mut_type_freqs)

    mut_type_freqs_pca_coords = mut_type_freqs.reset_index().assign(
        principal_component_1=pca_coords[:, 0],
        principal_component_2=pca_coords[:, 1],
        log10_total_count=lambda x: numpy.log(x["total_count"]) / numpy.log(10),
    )

    # percent variance explained by each component
    pca_var = 100 * pca.explained_variance_ratio_

    total_count_selection = alt.selection_single(
        fields=["log10_total_count"],
        init={
            "log10_total_count": numpy.log(synonymous_spectra_min_counts)
            / numpy.log(10)
        },
        bind=alt.binding_range(
            name="minimum log10 total counts",
            min=int(mut_type_freqs_pca_coords["log10_total_count"].min()),
            max=mut_type_freqs_pca_coords["log10_total_count"].max(),
        ),
    )

    clade_selection = alt.selection_multi(fields=["clade"], bind="legend")

    tooltip = ["clade", "total_count"]

    plot_size = 300  # scaled by component variance explained

    pca_chart = (
        alt.Chart(mut_type_freqs_pca_coords)
        .encode(
            y=alt.Y(
                "principal_component_1",
                title=f"PC1 ({pca_var[0]:.0f}% variance)",
                scale=alt.Scale(nice=False, padding=10),
                axis=alt.Axis(labels=False, ticks=False),
            ),
            x=alt.X(
                "principal_component_2",
                title=f"PC2 ({pca_var[1]:.0f}% variance)",
                scale=alt.Scale(nice=False, padding=10),
                axis=alt.Axis(labels=False, ticks=False),
            ),
            color=alt.Color(
                "clade",
                scale=alt.Scale(
                    scheme="viridis",
                    domain=sorted(mut_type_freqs_pca_coords["clade"].unique()),
                ),
            ),
            strokeWidth=alt.condition(clade_selection, alt.value(1.5), alt.value(0)),
            opacity=alt.condition(clade_selection, alt.value(0.9), alt.value(0.45)),
            size=alt.condition(clade_selection, alt.value(65), alt.value(45)),
        )
        .mark_point(filled=True, stroke="black")
        .add_selection(total_count_selection, clade_selection)
        .transform_filter(
            total_count_selection.log10_total_count <= alt.datum.log10_total_count
        )
        .configure_axis(grid=False)
        .configure_legend(columns=2)
        .properties(
            height=plot_size,
            width=plot_size * pca_var[1] / pca_var[0],
            title=title,
        )
    )

    if len(subsets) > 1:
        subset_selection = alt.selection_multi(fields=["subset"], bind="legend")
        pca_chart = (
            pca_chart.encode(
                shape=alt.Shape(
                    "subset", sort=subset_order, scale=alt.Scale(domain=subset_order)
                )
            )
            .add_selection(subset_selection)
            .transform_filter(subset_selection)
        )
        tooltip.append("subset")

    if partition:
        partition_selection = alt.selection_multi(fields=["partition"], bind="legend")
        pca_chart = (
            pca_chart.encode(
                shape=alt.Shape(
                    "partition",
                    scale=alt.Scale(
                        domain=mut_type_freqs_pca_coords["partition"].unique()
                    ),
                ),
            )
            .add_selection(partition_selection)
            .transform_filter(partition_selection)
        )
        tooltip.append("partition")

    if exclude_top:
        exclude_selection = alt.selection_multi(fields=["excluded"], bind="legend")
        pca_chart = (
            pca_chart.encode(
                shape=alt.Shape(
                    "excluded",
                    title=f"excluding top {exclude_top_n} per clade",
                    scale=alt.Scale(
                        domain=mut_type_freqs_pca_coords["excluded"].unique()
                    ),
                ),
            )
            .add_selection(exclude_selection)
            .transform_filter(exclude_selection)
        )
        tooltip.append(
            alt.Tooltip("excluded", title=f"excluding top {exclude_top_n} per clade")
        )

    pca_chart = pca_chart.encode(tooltip=tooltip)

    display(pca_chart)
    print("\n\n")

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

Compute statistical significance of differences between clades.
We just do this on "all" sequences for a clade, not partitioning the genomes:

In [10]:
all_mut_type_counts = (
    mut_type_counts.query("subset == 'all'")
    .drop(columns="subset")
    .assign(total_count=lambda x: x.groupby("clade")["count"].transform("sum"))
    .query("total_count >= @synonymous_spectra_min_counts")
    .drop(columns="total_count")
)

wide_all_mut_type_counts = all_mut_type_counts.pivot_table(
    index="mut_type",
    columns="clade",
    values="count",
    fill_value=0,
)

Now run chi2 test.
Also, Bonferroni correct the P-values (this is conservative, but is fine as these P-values are so tiny):

In [11]:
min_p = 1e-20  # plot P-values less than this as this

records = []
for clade1, clade2 in itertools.combinations(wide_all_mut_type_counts.columns, 2):
    chi2, p, dof, _ = scipy.stats.chi2_contingency(
        wide_all_mut_type_counts[[clade1, clade2]]
    )
    records.append((clade1, clade2, p, chi2))

chi2_stats = pd.DataFrame(records, columns=["clade_1", "clade_2", "p", "chi2"]).assign(
    p=lambda x: x["p"].clip(lower=min_p),
    bonferroni_p=lambda x: (x["p"] * len(x)).clip(upper=1),
)

Plot the Bonferroni corrected P-values.
Note since counts are very large, many comparisons will be highly significant:

In [12]:
p_chart = (
    alt.Chart(chi2_stats)
    .encode(
        x=alt.X("clade_1", title=None),
        y=alt.Y("clade_2", title=None),
        fill=alt.Fill(
            "bonferroni_p",
            title="Bonferroni corrected P-value",
            scale=alt.Scale(type="log", scheme="yelloworangered", reverse=True),
            legend=alt.Legend(orient="top"),
        ),
        tooltip=[
            "clade_1",
            "clade_2",
            alt.Tooltip("p", format=".2g"),
            alt.Tooltip("bonferroni_p", format=".2g"),
            alt.Tooltip("chi2", format=".2g"),
        ],
    )
    .mark_rect(stroke="black")
    .properties(width=alt.Step(14), height=alt.Step(14))
)

p_chart

alt.Chart(...)

Now get the mutation counts and fractions for each clade, both with and without excluding top mutations.
Then also compute a normalized **rate** for each mutation type, which is the fraction of all mutations that are of that type divided by the overall fraction of 4-fold synonymous sites sites that are the parental nucleotide in the mutation.

Note that a **caveat** is that for these rates we do not adjust the composition to account for any excluded 4-fold synonymous sites.
This is probably not currently a big problem, but could become a concern if a lot of sites are excluded:

In [13]:
parent_composition = (
    pd.read_csv(clade_founder_nts_csv)
    .query("four_fold_degenerate")
    .groupby(["clade", "nt"], as_index=False)
    .aggregate(parent_nt_count=pd.NamedAgg("site", "nunique"))
    .assign(
        parent_nt_frac=lambda x: (
            x["parent_nt_count"]
            / x.groupby("clade")["parent_nt_count"].transform("sum")
        ),
    )
)

mut_type_count_frac_rate = (
    pd.concat(
        [
            mut_type_counts.query("subset == 'all'")
            .query("partition == 'all'")[["clade", "mut_type", "count"]]
            .assign(exclude_top_mutations="no"),
            mut_type_counts_exclude_top.assign(exclude_top_mutations="yes"),
        ]
    )
    .assign(
        total_count=lambda x: x.groupby(["clade", "exclude_top_mutations"])[
            "count"
        ].transform("sum"),
        fraction=lambda x: x["count"] / x["total_count"],
        parent_nt=lambda x: x["mut_type"].str[0],
    )
    .query("total_count >= @synonymous_spectra_min_counts")
    .merge(
        parent_composition[["clade", "nt", "parent_nt_frac"]].rename(
            columns={"nt": "parent_nt"}
        ),
        how="left",
        validate="many_to_one",
    )
    .assign(rate=lambda x: x["fraction"] / x["parent_nt_frac"])
)

Plot the rates. The chart below is interactive, and you can click to select mutation types, clades, etc:

In [14]:
melted_df = (
    mut_type_count_frac_rate.assign(
        mut_type=lambda x: x["mut_type"].str.replace("to", " -> ")
    )
    .melt(
        id_vars=["clade", "mut_type", "exclude_top_mutations", "count"],
        value_vars=["fraction", "rate"],
    )
    .assign(
        variable=lambda x: x["variable"].map(
            {
                "fraction": "fraction of all mutations",
                "rate": "relative rate of mutation",
            }
        ),
        clade=lambda x: x["clade"].map(lambda c:
            f"{c} ({clade_synonyms[c]})" if c in clade_synonyms else c
        ),
    )
)

mut_type_selection = alt.selection_multi(fields=["mut_type"])

mut_type_selection_bar = (
    alt.Chart(melted_df[["mut_type"]].drop_duplicates())
    .encode(
        x=alt.X("mut_type", title="click / shift-click to select mutation types"),
        color=alt.condition(
            mut_type_selection, alt.value("darkgray"), alt.value("white")
        ),
    )
    .mark_rect(stroke="black")
    .add_selection(mut_type_selection)
    .properties(width=alt.Step(15))
)

frac_rate_chart_base = (
    alt.Chart(melted_df)
    .encode(
        x=alt.X("mut_type", title="mutation type"),
        y=alt.Y("value", title=None),
        color=alt.Color("clade", scale=alt.Scale(scheme="viridis")),
        strokeWidth=alt.condition(clade_selection, alt.value(0.5), alt.value(0)),
        opacity=alt.condition(clade_selection, alt.value(0.9), alt.value(0.2)),
        size=alt.condition(clade_selection, alt.value(50), alt.value(40)),
        column=alt.Column(
            "variable",
            title=None,
            header=alt.Header(
                labelOrient="left", labelFontSize=11, labelFontStyle="bold"
            ),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if melted_df[c].dtype == float else c
            for c in melted_df.columns.tolist()
        ],
    )
    .mark_point(filled=True, stroke="black")
    .add_selection(clade_selection, mut_type_selection)
    .transform_filter(mut_type_selection)
    .resolve_scale(y="independent")
    .properties(width=alt.Step(15), height=200)
)

frac_rate_chart = (
    (frac_rate_chart_base & mut_type_selection_bar)
    .configure_axis(grid=False)
    .configure_legend(columns=2)
    .configure(padding=20)
)

print(f"Saving to {rates_plot}")
frac_rate_chart.save(rates_plot)

frac_rate_chart

Saving to ../results/synonymous_mut_rates/rates_plot.html


alt.VConcatChart(...)

Get rates to write to file.
Get the rates by clade **without** excluding the top mutations:

In [15]:
rates_by_clade = mut_type_count_frac_rate.query("exclude_top_mutations == 'no'").drop(
    columns="exclude_top_mutations"
)

print(f"Writing rates to {rates_by_clade_csv}")
rates_by_clade.to_csv(rates_by_clade_csv, index=False, float_format="%.5g")

rates_by_clade

Writing rates to ../results/synonymous_mut_rates/rates_by_clade.csv


,clade,mut_type,count,total_count,fraction,parent_nt,parent_nt_frac,rate
0,20A,AtoC,285,17349,0.016427,A,0.289561,0.056732
1,20A,AtoG,1639,17349,0.094472,A,0.289561,0.326260
2,20A,AtoT,531,17349,0.030607,A,0.289561,0.105701
3,20A,CtoA,349,17349,0.020116,C,0.135129,0.148869
4,20A,CtoG,127,17349,0.007320,C,0.135129,0.054173
...,...,...,...,...,...,...,...,...
199,22F,GtoC,157,24997,0.006281,G,0.065280,0.096213
200,22F,GtoT,1578,24997,0.063128,G,0.065280,0.967031
201,22F,TtoA,885,24997,0.035404,T,0.510760,0.069317
202,22F,TtoC,5513,24997,0.220546,T,0.510760,0.431800


Get rates in different genome regions:

In [35]:
gene_type_parent_counts = (
    pd.read_csv(clade_founder_nts_csv)
    .query("gene in ['ORF1ab', 'M', 'N', 'S', 'E']")
    .assign(
        gene_type=lambda x: x["gene"].map(lambda p: "nonstructural" if p == "ORF1ab" else "structural")
    )
    .query("four_fold_degenerate")
    .groupby(["nt", "gene_type"], as_index=False)
    .aggregate(parent_count=pd.NamedAgg("site", "count"))
)

gene_type_parent_counts

,nt,gene_type,parent_count
0,A,nonstructural,7357
1,A,structural,5072
2,C,nonstructural,3017
3,C,structural,2883
4,G,nonstructural,1511
5,G,structural,1325
6,T,nonstructural,11582
7,T,structural,8856


In [38]:
gene_type_rates = (
    mutation_counts
    .query("protein in ['ORF1ab', 'M', 'N', 'S', 'E']")
    .query("not exclude")
    .query("four_fold_degenerate")
    .assign(
        gene_type=lambda x: x["protein"].map(lambda p: "nonstructural" if p == "ORF1ab" else "structural")
    )
    .groupby(["gene_type", "mut_type"], as_index=False)
    .aggregate({"count": "sum"})
    .assign(nt=lambda x: x["mut_type"].str[0])
    .merge(gene_type_parent_counts, validate="many_to_one")
    .assign(
        rate=lambda x: x["count"] / x["parent_count"],
        relative_rate=lambda x: x["rate"] / x["rate"].max(),
    )
    .sort_values("relative_rate", ascending=False)
)

gene_type_rates

,gene_type,mut_type,count,nt,parent_count,rate,relative_rate
5,nonstructural,CtoT,133682,C,3017,44.309579,1.000000
17,structural,CtoT,110054,C,2883,38.173430,0.861516
20,structural,GtoT,45648,G,1325,34.451321,0.777514
8,nonstructural,GtoT,38389,G,1511,25.406353,0.573383
18,structural,GtoA,19842,G,1325,14.975094,0.337965
6,nonstructural,GtoA,17523,G,1511,11.596956,0.261726
22,structural,TtoC,57154,T,8856,6.453704,0.145650
1,nonstructural,AtoG,46529,A,7357,6.324453,0.142733
13,structural,AtoG,30123,A,5072,5.939077,0.134036
10,nonstructural,TtoC,64712,T,11582,5.587291,0.126097


In [44]:
(
    alt.Chart(gene_type_rates)
    .encode(
        x=alt.X("mut_type", title="mutation type"),
        y=alt.Y("relative_rate", title="relative rate"),
        color=alt.Color("gene_type", title="gene type"),
        shape=alt.Color("gene_type"),
    )
    .mark_point(filled=True, size=50)
    .properties(width=250, height=150)
)

alt.Chart(...)